In [206]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import os
import sys
sys.path.append('..')
from Utils import eda_helper
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
import seaborn as sns

In [207]:
train= pd.read_csv('../dataset/train.csv')
test= pd.read_csv('../dataset/test.csv')

In [208]:
train

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,BlendProperty1,BlendProperty2,BlendProperty3,BlendProperty4,BlendProperty5,BlendProperty6,BlendProperty7,BlendProperty8,BlendProperty9,BlendProperty10
0,0.21,0.00,0.42,0.25,0.12,-0.021782,1.981251,0.020036,0.140315,1.032029,...,0.489143,0.607589,0.321670,-1.236055,1.601132,1.384662,0.305850,0.193460,0.580374,-0.762738
1,0.02,0.33,0.19,0.46,0.00,-0.224339,1.148036,-1.107840,0.149533,-0.354000,...,-1.257481,-1.475283,-0.437385,-1.402911,0.147941,-1.143244,-0.439171,-1.379041,-1.280989,-0.503625
2,0.08,0.08,0.18,0.50,0.16,0.457763,0.242591,-0.922492,0.908213,0.972003,...,1.784349,0.450467,0.622687,1.375614,-0.428790,1.161616,0.601289,0.872950,0.660000,2.024576
3,0.25,0.42,0.00,0.07,0.26,-0.577734,-0.930826,0.815284,0.447514,0.455717,...,-0.066422,0.483730,-1.865442,-0.046295,-0.163820,-0.209693,-1.840566,0.300293,-0.351336,-1.551914
4,0.26,0.16,0.08,0.50,0.00,0.120415,0.666268,-0.626934,2.725357,0.392259,...,-0.118913,-1.172398,0.301785,-1.787407,-0.493361,-0.528049,0.286344,-0.265192,0.430513,0.735073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.50,0.12,0.00,0.26,0.12,0.279523,-0.054170,-0.391227,0.400222,1.032029,...,-0.028366,-0.327297,-0.316933,-1.294092,-0.530259,-0.421526,-0.320869,0.709627,-0.737244,-0.744289
1996,0.19,0.31,0.00,0.37,0.13,-0.887185,0.610050,0.178606,1.083154,-2.822749,...,-0.449245,0.156778,-0.367445,-0.938615,-0.577451,-0.209996,-0.370505,-0.195531,-0.032834,0.269718
1997,0.38,0.06,0.14,0.31,0.11,0.568978,-0.196759,-0.646318,-0.980070,1.032029,...,0.029135,0.164890,-0.092942,-1.134490,-0.437479,-0.695636,-0.101073,0.063650,0.624368,-0.477053
1998,0.50,0.16,0.00,0.18,0.16,-0.067453,0.321977,-0.137535,0.238507,0.017455,...,-0.232960,-0.464947,0.112536,-0.793522,-0.811272,-1.194914,0.100644,0.760116,-0.751394,-0.857598


In [209]:

# --- Step 1: Assume `train` DataFrame is already loaded ---
# For demonstration purposes, I will create a synthetic `train` DataFrame.
# In your actual script, you will already have `train` loaded.


# Separate features (X) and targets (y) from your `train` DataFrame
feature_columns = [col for col in train.columns if 'Component' in col]
target_columns = [col for col in train.columns if 'BlendProperty' in col]

X_train = train[feature_columns]
y_train = train[target_columns]
# Separate features (X) from your `test` DataFrame
X_test = test[feature_columns]

In [210]:
test.shape

(500, 56)

In [213]:
import pandas as pd
import numpy as np
from scipy.stats import entropy

def create_advanced_features(df):
    """
    Advanced feature engineering for fluid mixing prediction
    
    Args:
        df: DataFrame with columns Component1_fraction, Component2_fraction, etc.
            and Component1_Property1, Component2_Property1, etc.
    
    Returns:
        DataFrame with original columns plus engineered features
    """
    
    # Create a copy to avoid modifying original
    df_new = df.copy()
    
    # Extract component fractions and properties
    fraction_cols = [col for col in df.columns 
                 if 'fraction' in col.lower() and 'prop' not in col.lower()]
    target_columns = [col for col in train.columns if 'BlendProperty' in col]
    property_cols = train.columns.drop(fraction_cols).drop(target_columns).to_list()
    # Get unique property names (Property1, Property2, etc.)
    properties = list(set([col.split('_')[-1] for col in property_cols]))
    print(len(fraction_cols),fraction_cols)
    print(len(property_cols), property_cols)
    print(properties)
    # Get number of components
    n_components = len(fraction_cols)
    
    print(f"Found {n_components} components and {len(properties)} properties")
    
    # # ===========================================
    # # 1. MIXTURE COMPLEXITY FEATURES
    # # ===========================================
    
    # # Mixture entropy: Measures how evenly distributed the components are
    # # Higher entropy = more uniform mixture, Lower entropy = dominated by few components
    fractions = df[fraction_cols].values
    # df_new['mixture_entropy'] = -np.sum(fractions * np.log(fractions + 1e-10), axis=1)
    
    # # Effective number of components: How many components are "really" contributing
    # # Value close to 1 = dominated by one component, close to n_components = all contribute equally
    # df_new['effective_components'] = 1 / np.sum(fractions**2, axis=1)
    
    # # Fraction standard deviation: How much variation in component amounts
    # # High std = uneven mixture, Low std = balanced mixture
    # df_new['fraction_std'] = np.std(fractions, axis=1)
    
    # # ===========================================
    # # 2. DOMINANT COMPONENT FEATURES
    # # ===========================================
    
    # # Identify dominant component and its properties
    # df_new['max_fraction'] = np.max(fractions, axis=1)
    # df_new['second_max_fraction'] = np.partition(fractions, -2, axis=1)[:, -2]
    
    # # Dominance ratio: How much more dominant is the top component vs second
    # # Close to 1 = two components nearly equal, much higher = one clearly dominates
    # df_new['dominance_ratio'] = df_new['max_fraction'] / (df_new['second_max_fraction'] + 1e-10)
    
    # # Which component is dominant (categorical feature)
    # df_new['dominant_component'] = np.argmax(fractions, axis=1)
    
    # ===========================================
    # 3. CROSS-COMPONENT INTERACTIONS
    # ===========================================
    
    # Pairwise fraction interactions: How components interact with each other
    # Captures synergistic/antagonistic effects between component pairs
    # for i in range(n_components):
    #     for j in range(i+1, n_components):
    #         col_i = fraction_cols[i]
    #         col_j = fraction_cols[j]
    #         df_new[f'interaction_{i+1}_{j+1}'] = df[col_i] * df[col_j]
    
    # ===========================================
    # 4. WEIGHTED PROPERTY STATISTICS
    # ===========================================
    
    for prop in properties:
        # Get property columns for this property
        prop_cols = [col for col in property_cols if col.endswith(prop)]
        
        if len(prop_cols) == n_components:
            prop_values = df[prop_cols].values
            
            # Weighted average: Traditional mixing rule (fraction × property)
            # This is the baseline assumption for most mixing scenarios
            df_new[f'weighted_avg_{prop}'] = np.sum(fractions * prop_values, axis=1)
            
            # Weighted variance: How much the property values vary in the mixture
            # High variance = components have very different properties
            weighted_avg = df_new[f'weighted_avg_{prop}'].values.reshape(-1, 1)
            df_new[f'weighted_var_{prop}'] = np.sum(fractions * (prop_values - weighted_avg)**2, axis=1)
            
            # Weighted standard deviation
            df_new[f'weighted_std_{prop}'] = np.sqrt(df_new[f'weighted_var_{prop}'])
            
            # Property range in mixture: Max - Min weighted property
            # Shows the span of property values present in the mixture
            weighted_props = fractions * prop_values
            df_new[f'weighted_range_{prop}'] = np.max(weighted_props, axis=1) - np.min(weighted_props, axis=1)
            
            # Weighted skewness: Asymmetry in property distribution
            # Positive = higher values dominate, Negative = lower values dominate
            centered = prop_values - weighted_avg
            df_new[f'weighted_skew_{prop}'] = np.sum(fractions * centered**3, axis=1) / (df_new[f'weighted_std_{prop}']**3 + 1e-10)
    
    # ===========================================
    # 5. NON-LINEAR MIXING RULES
    # ===========================================
    
    # for prop in properties:
    #     prop_cols = [col for col in property_cols if col.endswith(prop)]
        
    #     if len(prop_cols) == n_components:
    #         prop_values = df[prop_cols].values
            
    #         # Quadratic mixing: For properties that don't mix linearly
    #         # Common in physical systems where interactions are non-linear
    #         df_new[f'quadratic_mix_{prop}'] = np.sum(fractions * prop_values**2, axis=1)
            
    #         # Logarithmic mixing: For properties that mix multiplicatively
    #         # Common in chemical systems (e.g., viscosity, reaction rates)
    #         log_props = np.log(np.abs(prop_values) + 1e-10)
    #         df_new[f'log_mix_{prop}'] = np.exp(np.sum(fractions * log_props, axis=1))
            
    #         # Harmonic mixing: For properties like thermal conductivity
    #         # Weighted harmonic mean
    #         df_new[f'harmonic_mix_{prop}'] = 1 / np.sum(fractions / (prop_values + 1e-10), axis=1)
    
    # ===========================================
    # 6. PROPERTY CONTRAST FEATURES
    # ===========================================
    
    # for prop in properties:
    #     prop_cols = [col for col in property_cols if col.endswith(prop)]
        
    #     if len(prop_cols) == n_components:
    #         prop_values = df[prop_cols].values
            
    #         # Property of dominant component
    #         dominant_indices = np.argmax(fractions, axis=1)
    #         df_new[f'dominant_{prop}'] = prop_values[np.arange(len(prop_values)), dominant_indices]
            
    #         # Property contrast: Difference between max and min property values
    #         # High contrast = very different components mixed together
    #         df_new[f'property_contrast_{prop}'] = np.max(prop_values, axis=1) - np.min(prop_values, axis=1)
            
    #         # Relative property contribution: How much each component contributes to final property
    #         # Shows which component is most influential for this property
    #         weighted_props = fractions * prop_values
    #         total_weighted = np.sum(weighted_props, axis=1).reshape(-1, 1)
    #         relative_contrib = weighted_props / (total_weighted + 1e-10)
    #         df_new[f'max_contrib_{prop}'] = np.max(relative_contrib, axis=1)
    
    # ===========================================
    # 7. ADVANCED INTERACTION FEATURES
    # ===========================================
    
    # Cross-property interactions: How different properties interact
    # Captures complex physical/chemical interactions between properties
    # for i, prop1 in enumerate(properties):
    #     for j, prop2 in enumerate(properties):
    #         if i < j:  # Avoid duplicate pairs
    #             prop1_cols = [col for col in property_cols if col.endswith(prop1)]
    #             prop2_cols = [col for col in property_cols if col.endswith(prop2)]
                
    #             if len(prop1_cols) == n_components and len(prop2_cols) == n_components:
    #                 prop1_values = df[prop1_cols].values
    #                 prop2_values = df[prop2_cols].values
                    
    #                 # Weighted correlation between properties
    #                 weighted_prop1 = np.sum(fractions * prop1_values, axis=1)
    #                 weighted_prop2 = np.sum(fractions * prop2_values, axis=1)
    #                 df_new[f'cross_product_{prop1}_{prop2}'] = weighted_prop1 * weighted_prop2
    
    # ===========================================
    # 8. FRACTION-BASED POLYNOMIAL FEATURES
    # ===========================================
    
    # Quadratic fraction terms: For non-linear fraction effects
    # Captures diminishing returns or acceleration effects
    # for i, col in enumerate(fraction_cols):
    #     df_new[f'{col}_squared'] = df[col]**2
    #     df_new[f'{col}_sqrt'] = np.sqrt(df[col])
    
    # ===========================================
    # 9. MIXTURE STABILITY INDICATORS
    # ===========================================
    
    # Gini coefficient for fractions: Inequality measure
    # 0 = perfectly equal mixture, 1 = completely dominated by one component
    # def gini_coefficient(x):
    #     sorted_x = np.sort(x)
    #     n = len(x)
    #     index = np.arange(1, n + 1)
    #     return (2 * np.sum(index * sorted_x)) / (n * np.sum(sorted_x)) - (n + 1) / n
    
    # df_new['fraction_gini'] = np.apply_along_axis(gini_coefficient, 1, fractions)
    
    print(f"Created {len(df_new.columns) - len(df.columns)} new features")
    print(f"New feature categories:")
    print("- Mixture complexity: entropy, effective_components, fraction_std")
    print("- Dominant component: max_fraction, dominance_ratio, dominant_component")
    print("- Cross-interactions: pairwise fraction interactions")
    print("- Weighted statistics: weighted_avg, weighted_var, weighted_std, weighted_range, weighted_skew")
    print("- Non-linear mixing: quadratic_mix, log_mix, harmonic_mix")
    print("- Property contrasts: dominant_property, property_contrast, max_contrib")
    print("- Cross-property interactions: cross_product between different properties")
    print("- Polynomial fractions: squared and sqrt transformations")
    print("- Stability indicators: fraction_gini")
    
    return df_new

# Usage:
# train_engineered = create_advanced_features(train)
# test_engineered = create_advanced_features(test)

In [214]:
train_new=create_advanced_features(train)

5 ['Component1_fraction', 'Component2_fraction', 'Component3_fraction', 'Component4_fraction', 'Component5_fraction']
50 ['Component1_Property1', 'Component2_Property1', 'Component3_Property1', 'Component4_Property1', 'Component5_Property1', 'Component1_Property2', 'Component2_Property2', 'Component3_Property2', 'Component4_Property2', 'Component5_Property2', 'Component1_Property3', 'Component2_Property3', 'Component3_Property3', 'Component4_Property3', 'Component5_Property3', 'Component1_Property4', 'Component2_Property4', 'Component3_Property4', 'Component4_Property4', 'Component5_Property4', 'Component1_Property5', 'Component2_Property5', 'Component3_Property5', 'Component4_Property5', 'Component5_Property5', 'Component1_Property6', 'Component2_Property6', 'Component3_Property6', 'Component4_Property6', 'Component5_Property6', 'Component1_Property7', 'Component2_Property7', 'Component3_Property7', 'Component4_Property7', 'Component5_Property7', 'Component1_Property8', 'Component2_Pr

In [215]:
train_new

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,weighted_avg_Property5,weighted_var_Property5,weighted_std_Property5,weighted_range_Property5,weighted_skew_Property5,weighted_avg_Property8,weighted_var_Property8,weighted_std_Property8,weighted_range_Property8,weighted_skew_Property8
0,0.21,0.00,0.42,0.25,0.12,-0.021782,1.981251,0.020036,0.140315,1.032029,...,0.301416,0.758290,0.870798,0.616867,0.880964,-0.330932,0.313036,0.559496,0.447258,0.219749
1,0.02,0.33,0.19,0.46,0.00,-0.224339,1.148036,-1.107840,0.149533,-0.354000,...,-0.534043,1.788622,1.337394,1.237351,0.088187,-0.207363,0.578388,0.760518,0.653462,0.583079
2,0.08,0.08,0.18,0.50,0.16,0.457763,0.242591,-0.922492,0.908213,0.972003,...,-1.673519,1.068607,1.033735,1.082770,0.507241,0.039688,1.534447,1.238728,0.878361,0.678077
3,0.25,0.42,0.00,0.07,0.26,-0.577734,-0.930826,0.815284,0.447514,0.455717,...,0.454384,0.217863,0.466758,0.336918,-0.229753,0.825966,2.209988,1.486603,1.318843,-0.306742
4,0.26,0.16,0.08,0.50,0.00,0.120415,0.666268,-0.626934,2.725357,0.392259,...,0.784656,0.371424,0.609445,0.683761,-0.606605,0.447709,1.012846,1.006403,0.913220,-0.200693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.50,0.12,0.00,0.26,0.12,0.279523,-0.054170,-0.391227,0.400222,1.032029,...,0.065456,0.910578,0.954242,0.719447,0.404029,0.516183,2.362044,1.536894,1.411772,-0.449501
1996,0.19,0.31,0.00,0.37,0.13,-0.887185,0.610050,0.178606,1.083154,-2.822749,...,-0.092031,0.490803,0.700573,0.491904,-0.233853,0.463288,0.488689,0.699063,0.486632,0.119388
1997,0.38,0.06,0.14,0.31,0.11,0.568978,-0.196759,-0.646318,-0.980070,1.032029,...,-1.192568,0.810180,0.900100,0.701541,0.924387,0.278017,0.867741,0.931526,0.477888,0.307438
1998,0.50,0.16,0.00,0.18,0.16,-0.067453,0.321977,-0.137535,0.238507,0.017455,...,-0.587432,0.343070,0.585722,0.497296,1.632090,0.795131,1.093767,1.045833,1.016137,-0.219762


In [216]:
test_new=create_advanced_features(test)

5 ['Component1_fraction', 'Component2_fraction', 'Component3_fraction', 'Component4_fraction', 'Component5_fraction']
50 ['Component1_Property1', 'Component2_Property1', 'Component3_Property1', 'Component4_Property1', 'Component5_Property1', 'Component1_Property2', 'Component2_Property2', 'Component3_Property2', 'Component4_Property2', 'Component5_Property2', 'Component1_Property3', 'Component2_Property3', 'Component3_Property3', 'Component4_Property3', 'Component5_Property3', 'Component1_Property4', 'Component2_Property4', 'Component3_Property4', 'Component4_Property4', 'Component5_Property4', 'Component1_Property5', 'Component2_Property5', 'Component3_Property5', 'Component4_Property5', 'Component5_Property5', 'Component1_Property6', 'Component2_Property6', 'Component3_Property6', 'Component4_Property6', 'Component5_Property6', 'Component1_Property7', 'Component2_Property7', 'Component3_Property7', 'Component4_Property7', 'Component5_Property7', 'Component1_Property8', 'Component2_Pr

In [217]:
test_new.shape

(500, 106)

In [218]:
train_new.isna().sum().sum()

0

In [219]:
# Create multiplicative features for each component
for i in range(1, 6):  # for each component (1-5)
    component_fraction = f'Component{i}_fraction'
    for j in range(1, 11):  # for each property (1-10)
        component_property = f'Component{i}_Property{j}'
        new_feature_name = f'Component{i}_Prop_{j}_x_Fraction'

        train_new[new_feature_name] = train_new[component_fraction] * train_new[component_property]
        test_new[new_feature_name] = test_new[component_fraction] * test_new[component_property]

print(f"New shape of train after multiplicative features: {train_new.shape}")

New shape of train after multiplicative features: (2000, 165)


/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_24147/2942112777.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_new[new_feature_name] = test_new[component_fraction] * test_new[component_property]
/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_24147/2942112777.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_new[new_feature_name] = train_new[component_fraction] * train_new[component_property]
/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_24147/2942112777.py:9: PerformanceWarn

In [220]:
train_new.shape

(2000, 165)

In [221]:
test_new.shape

(500, 156)

In [222]:
train_new.to_csv('train_ultra_featured.csv', index=False)
test_new.to_csv('test_ultra_featured.csv', index=False)